# **OJK ChatBot - LangChain**

## **Setup**

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.questions import get_question
from utils.model_config import ModelName, get_model

config = get_config()

In [3]:
STORE = False
DELETE = False
TOP_K = 10
TOP_N = 6
model_name = ModelName.AZURE_OPENAI
query_str = get_question("k")
# query_str = "Apa judul peraturan nomor 37 /SEOJK.03/2016?"

## **Define Model**

In [4]:
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Indexing**

### **Load**

In [5]:
from utils.documents_text_extract import extract_all_documents_in_directory

documents_dir = './data/documents'
metadata_path = './data/metadata/files_metadata.csv'

if STORE:
    documents = extract_all_documents_in_directory(documents_dir, metadata_path, treshold=0.98)


### **Split**

In [6]:
from utils.document_split import document_splitter

if STORE:
    all_splits = document_splitter(docs=documents)

### **Storing**

In [7]:
from utils.vector_store import PineconeIndexManager

pinecone = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)

if STORE:
    pinecone.store_vector_index(docs=all_splits, delete=DELETE)
    vector_store = pinecone.load_vector_index()
else: 
    vector_store = pinecone.load_vector_index()

## **Retrieval and Generation**

### **Retrieve**

In [8]:
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

# retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})
compressor = CohereRerank(cohere_api_key=config['cohere_api_key'], top_n=TOP_N)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})
)

In [9]:
retrieved_docs = retriever.invoke(input=query_str)
retrieved_docs

[Document(metadata={'effective_date': '26 Februari 2008', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'page_number': 4.0, 'regulation_number': '36/PMK.010/2008', 'regulation_type': 'Klasifikasi Bapepam', 'sector': 'IKNB', 'subsector': 'Asuransi', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'relevance_score': 0.9850429}, page_content='2) Dalam hal pembayaran SWDKLLJ dilakukan setelah melewati batas\n seharusnya dibayar dengan ketentuan denda yang dikenakan paling\nbesar Rp100.000,00 (seratus ribu rupiah).\n(3) Dalam hal ketentuan mengenai batas waktu sebagaimana dimaksud\ngeografis daerah setempat, Direksi perusahaan yang ditunjuk untuk\nmenyelenggarakan Dana Kecelakaan Lalu Lintas Jalan diberi\nbesarnya denda SWDKLLJ, dengan ketentuan batas wak

### **Generate**

In [10]:
import json
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import (
    ConfigurableFieldSpec,
    RunnablePassthrough,
)

# from utils.chat_history import ChatHistory

from langchain_core.retrievers import BaseRetriever

# from utils.final_chain_input_type import InputTypeFinalChain

from langchain_core.runnables.history import RunnableWithMessageHistory


from utils.model_config import ModelName

In [11]:

def _format_metadata(metadata):
    """Remove filename from metadata."""
    # check if file_name is in metadata
    if "file_name" in metadata:
        metadata.pop("file_name", None)
    return metadata


def _combine_documents(docs):
    """Combine documents into a single JSON string."""
    doc_list = [{"metadata": _format_metadata(
        doc.metadata), "page_content": doc.page_content} for doc in docs]
    return json.dumps(doc_list, indent=2)

In [12]:
from utils.chat_history import ChatStore

# chat_history = ChatHistory(max_history_length=5)
K_STORE = 10
chat_store = ChatStore(k=K_STORE)

In [13]:
from utils.rag_chain_with_chat_history import create_chain_with_chat_history

_TEMPLATE = """Given the following conversation and a follow-up question, \
rephrase the follow-up question to be a standalone question in its original language. 
If the follow-up question is not clear, indicate so. 
If the chat history is not relevant to the follow-up question, please ignore the chat history.

Chat History:
{chat_history}

Follow-up Question: {question}
Standalone Question: """

_ANSWER_TEMPLATE = """The context information is below.
Context: 
{context}

Based on the context and the metadata information provided, \
answer the query related to banking compliance in Indonesia.
Use the context and metadata information only, without relying on prior knowledge. 
ALWAYS ANSWER IN THE USER'S LANGUAGE.

Please provide your answer in the following format, \
including the regulation number and file URL if available:

(Answer...) \n\n
Source: [metadata['regulation_number']](metadata['file_url'])

If you cannot find the regulation number, just provide the (Answer...). 
If the file_url ends with '.pdf', you can add the metadata['page_number'] in the URL like this: 

(Answer...) \n\n
Source: [metadata['regulation_number']](metadata['file_url#page=metadata['page_number']')

(Answer...) is the answer to the question, don't write '(Answer...)' in the answer.
DO NOT PROVIDE AMBIGUOUS ANSWERS.

Question: {question}
"""

# chain = create_chain_with_chat_history(
#     contextualize_q_prompt_str=_TEMPLATE,
#     qa_system_prompt_str=_ANSWER_TEMPLATE,
#     retriever=retriever,
#     llm_model=llm_model,
#     get_session_history=chat_store.get_session_history,
# )

In [14]:
CONTEXTUALIZE_Q_PROMPT_STR = _TEMPLATE
QA_SYSTEM_PROMPT_STR = _ANSWER_TEMPLATE
QA_PROMPT = ChatPromptTemplate.from_template(QA_SYSTEM_PROMPT_STR)
CONTEXTUALIZE_Q_PROMPT = PromptTemplate.from_template(CONTEXTUALIZE_Q_PROMPT_STR)
# _inputs_question = RunnableMap(
#     standalone_question=RunnablePassthrough.assign(
#         chat_history=lambda x: x["chat_history"].get_session_history()
#     )
#     | CONTEXTUALIZE_Q_PROMPT
#     | llm_model
#     | StrOutputParser(),
# )
_inputs_question = CONTEXTUALIZE_Q_PROMPT | llm_model | StrOutputParser()
# _context_chain = _inputs_question | itemgetter("question") 
# test_chain = _inputs_question | retriever | _combine_documents
_context_chain = _inputs_question | {
    "context": retriever | _combine_documents,
    "question": RunnablePassthrough()
} 
conversational_qa_with_context_chain = (
    # _inputs_question
    _context_chain
    | {
        "question": itemgetter("question"),
        "answer": QA_PROMPT | llm_model | StrOutputParser(),
        "context": itemgetter("context"),
    }
    # | StrOutputParser()
)
# final_chain = conversational_qa_with_context_chain.with_types(input_type=InputTypeFinalChain)

final_chain = RunnableWithMessageHistory(
    conversational_qa_with_context_chain,
    get_session_history=chat_store.get_session_history,
    input_messages_key="question",
    output_messages_key="answer",
    history_messages_key="chat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [15]:
response = final_chain.invoke(
    {"question": "Kalau mobil?"},
    config={
        "configurable": {"user_id": "xmriz", "conversation_id": "conv1"}
    },
)

In [16]:
response

{'question': 'Apakah Anda memiliki mobil?',
 'answer': '(Answer...) \nTidak ada informasi yang relevan dalam konteks yang menjawab pertanyaan Anda.',
 'context': '[\n  {\n    "metadata": {\n      "effective_date": "26 Februari 2008",\n      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf",\n      "page_number": 3.0,\n      "regulation_number": "36/PMK.010/2008",\n      "regulation_type": "Klasifikasi Bapepam",\n      "sector": "IKNB",\n      "subsector": "Asuransi",\n      "title": "Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan",\n      "relevance_score": 0.00095846487\n    },\n    "page_content": "Sepeda motor, sepeda kumbang dan scooter di atas 50 cc sampai\\n250 cc dan kendaraan bermotor roda tiga sebesar Rp 32.000,00\\n(tiga puluh dua ribu rupiah).\\nd.\\nPick

In [17]:
print(chat_store.get_session_history(user_id="xmriz", conversation_id="conv1"))

Human: Kalau mobil?
AI: (Answer...) 
Tidak ada informasi yang relevan dalam konteks yang menjawab pertanyaan Anda.


#### **Streaming**

In [18]:
# from utils.rag_chain_with_chat_history import get_response

# response = get_response(
#     chain=chain,
#     question=query_str,
#     user_id="xmriz",
#     conversation_id="conv123",
# )

# response

In [19]:
# from utils.rag_chain_with_chat_history import print_answer_stream

# await print_answer_stream(chain=chain, chat_history=chat_history, question=query_str)

#### **With Question, Answer, and Context**

In [20]:
# from utils.rag_chain_with_chat_history import get_response

# response = get_response(chat_history=chat_history, chain=chain, question="Kalau mobil?")
# response

In [21]:
# print(response['answer'])

In [22]:
# print(response['context'])

In [23]:
# print(chat_history.get_formatted_history())

## **Evaluation**

In [24]:
eval_questions = [
    "Apa itu SWDKLLJ?",
    "Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris dengan ketentuan apa?",
    "KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOMOR KEP-06/PM/1996 tentang apa?",
    "Siapa yang menandatangani keputusan ketua bapepam nomor kep-07/PM/1996?",
    "Penolakan atas permohonan persetujuan mengenai pengajuan atau perubahan peraturan Lembaga Kliring dan Penjaminan dilakukan dengan menggunakan formulir nomor berapa?",
    "Anggota Dewan Pengawas Syariah Perusahaan Pembiayaan dilarang untuk apa?",
    "Berapa uang pertanggungan dan pertanggungan total loss only (tlo) untuk kategori 5?",
    "Perusahaan wajib menyampaikan laporan hasil pengawasan Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 2 dalam bentuk apa, kepada siapa, dan paling lambat kapan?",
    "Batas tingkat solvabilitas minimum bagi perusahaan asuransi dan perusahaan reasuransi ditingkatkan berdasarkan apa?",
    "Kapan PERATURAN KETUA BADAN PENGAWAS PASAR MODAL DAN LEMBAGA KEUANGAN NOMOR: PER- 10/BL/2012 mulai berlaku?",
    "Apa itu Dana Tabarru’",
    "Dalam rangka mendorong pertumbuhan industri Dana Pensiun, dipandang perlu mengubah ketentuan mengenai sanksi administratif berupa apa?",
    "Laporan Tahunan dan Laporan Semester sekurang-kurangnya mencakup aspek apa saja?",
    "Pendiri harus mengajukan permohonan tertulis kepada siapa untuk mendapatkan pengesahan pendirian Dana Pensiun Lembaga Keuangan?",
    "Berapa tarif pemotongan pajak untuk penghasilan bruto di atas seratus juta sampai dengan dua ratus juta rupiah?",
    "Apa yang dimaksud dengan 'telah beroperasi'?",
    "Lembaga Jasa Keuangan Non-Bank yang tidak memenuhi ketentuan Pasal 2 dan Pasal 3 dalam Peraturan OJK ini, dikenakan sanksi administratif berupa apa saja?",
    "Apa yang dimaksud dengan Lembaga Sertifikasi Profesi?",
    "Apa pertimbangan profil risiko perusahaan?",
    "LJKNB yang memiliki total aset lebih dari Rp500.000.000.000,00 (lima ratus miliar rupiah) sampai dengan Rp1.000.000.000.000,00 (satu triliun rupiah) wajib untuk apa?",
    "Siapa presiden republik Indonesia yang menandatangani peraturan pemerintah republik Indonesia nomor 12 Tahun 2004?",
    "Berapa jumlah modal yang disetor bursa efek?",
    "Bagaimana norma pemeriksaan yang menyangkut pihak yang diperiksa?",
    "Apa isi Pasal II PERATURAN PEMERINTAH REPUBLIK INDONESIA NOMOR 63 TAHUN 1999?",
    "Berapa modal yang disetor bagi perusahaan Asuransi Jiwa yang seluruh pemiliknya warga negara Indonesia?",
    "Apa itu Bank Perkreditan Rakyat?",
    "Bagaimana pelaksanaan wawancara jika calon pemegang saham pengendali BPR adalah pemerintah?",
    "Berapa modal yang disetor untuk mendirikan Bank Umum?",
    "Bagaimana cara bank untuk meningkatkan good corporate governance?",
    "Peraturan Bank Indonesia Nomor 7/33/PBI/2005 berisi pencabutan atas peraturan bank indonesia nomor berapa dan tentang apa?",
    "Surat edaran bank indonesia no 15/14/DPNP ditujukan kepada siapa?",
    "Apa isi ketentuan peralihan dari surat edaran No. 15/20/DKBU?",
    "Surat edaran bank indonesia No. 15/22/DPbS mencabut surat edaran bank indonesia nomor berapa?",
    "Kapan dan di mana Surat Edaran No. 15/25/DPNP mulai berlaku?",
    "Bank Syariah wajib membentuk cadangan kerugian penurunan nilai (CKPN) atas apa?",
    "Berapa penurunan IHSG BEI sejak tanggal 20 Mei 2013-27 Agustus 2013?",
    "Berapa lama Pengguna S-INVEST yang bertindak sebagai Bank Kustodian melakukan uji coba penyampaian surat?",
    "Apa judul regulasi nomor 1/SEOJK.04/2002?",
    "Penilaian Faktor Profil Risiko, menjelaskan mengenai mekanisme penilaian faktor profil risiko, yang terdiri dari 9 jenis risiko, apa saja jenis risiko tersebut?",
    "Sumber daya merupakan kriteria dalam melakukan penilaian terhadap lembaga pemeringkat yang digunakan untuk apa?",
    "Asuransi dapat diberikan dalam bentuk apa saja?",
    "Badan Supervisi terdiri dari berapa anggota?",
    "Apa yang dimaksud dengan Dana Pensiun berdasarkan pasal 134 UU PPSK?",
    "Tahun berapa krisis keuangan terbesar di Amerika Serikat pernah terjadi?",
    "Siapa menteri keuangan yang mensahkan UU RI Nomor 84 Tahun 1958?"
]

eval_answers = [
    "Sumbangan Wajib Dana Kecelakaan Lalu Lintas adalah sumbangan wajib sebagaimana dimaksud dalam Undang-Undang Nomor 34 Tahun 1964 tentang Dana Kecelakaan Lalu Lintas Jalan juncto Peraturan Pemerintah Nomor 18 Tahun 1965 tentang Ketentuan-Ketentuan Pelaksanaan Dana Kecelakaan Lalu Lintas Jalan",
    "Bursa Efek wajib menyampaikan laporan realisasi anggaran kepada Bapepam melalui dewan komisaris, dengan ketentuan bahwa laporan tersebut disampaikan secara kumulatif triwulanan dan diterima oleh Bapepam selambat-lambatnya pada hari ke dua belas setelah berakhirnya triwulan yang bersangkutan",
    "TATA CARA PEMBERIAN PERSETUJUAN ANGGARAN DASAR BURSA EFEK",
    "I Putu Gede Ary Suta, Badan Pengawas Pasar Modal",
    "Penolakan atas permohonan persetujuan mengenai pengajuan atau perubahan peraturan Lembaga Kliring dan Penjaminan dilakukan dengan menggunakan Formulir Nomor III.B.2-3 lampiran 3 peraturan ini.",
    "Dilarang melakukan rangkap jabatan sebagai anggota direksi atau dewan komisaris pada Perusahaan Pembiayaan, Dilarang merangkap jabatan sebagai anggota Dewan Pengawas Syariah lebih dari 2 (dua) Perusahaan Pembiayaan lain.",
    "Lebih dari Rp800.0000.000,00 dan 0,28%",
    "Perusahaan wajib menyampaikan laporan hasil pengawasan Dewan Pengawas Syariah sebagaimana dimaksud dalam Pasal 2 dalam bentuk dokumen fisik (hardcopy) dan format digital (softcopy) kepada Biro Perasuransian Badan Pengawas Pasar Modal dan Lembaga Keuangan, paling lambat tanggal 31 Maret tahun berikutnya.",
    "Batas tingkat solvabilitas minimum bagi perusahaan asuransi dan perusahaan reasuransi sebagaimana dimaksud dalam Pasal 3 Keputusan Menteri Keuangan Nomor 424/KMK.06/2003 tentang Kesehatan Keuangan Perusahaan Asuransi dan Perusahaan Reasuransi sebagaimana telah beberapa kali diubah terakhir dengan Peraturan Menteri Keuangan Nomor 158/PMK.010/2008, ditetapkan berdasarkan besarnya risiko kerugian yang mungkin timbul sebagai akibat dari deviasi dalam pengelolaan kekayaan dan kewajiban.",
    "Peraturan Ketua ini mulai berlaku pada tanggal 1 Juni 2013.",
    "Dana Tabarru’ adalah kumpulan dana yang berasal dari kontribusi para peserta, yang mekanisme penggunaannya sesuai dengan Akad Tabarru’ yang disepakati",
    "Dalam rangka mendorong pertumbuhan industri Dana Pensiun, dipandang perlu mengubah ketentuan mengenai sanksi administratif berupa denda atas keterlambatan penyampaian laporan investasi sebagaimana telah ditetapkan dalam Peraturan Menteri Keuangan Nomor 199/PMK.010/2008 tentang Investasi Dana Pensiun",
    "Operasional, keuangan, investasi, aktuaria, dan statistik",
    "Pendiri harus mengajukan permohonan tertulis kepada Menteri c.q. Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan u.p. Kepala Biro Dana Pensiun, Badan Pengawas Pasar Modal dan Lembaga Keuangan untuk mendapatkan pengesahan pendirian Dana Pensiun Lembaga Keuangan",
    "15%",
    "Yang dimaksud dengan “telah beroperasi” adalah telah mendapatkan izin usaha bagi perusahaan asuransi jiwa, perusahaan asuransi umum, perusahaan reasuransi, dan lembaga penjaminan termasuk yang menyelenggarakan seluruh atau sebagian usahanya dengan prinsip syariah, atau telah memperoleh pengesahan bagi dana pensiun pemberi kerja.",
    "Peringatan tertulis, penilaian kembali kemampuan dan kepatutan, larangan menjadi pemegang saham",
    "Lembaga Sertifikasi Profesi adalah lembaga pelaksana kegiatan sertifikasi profesi yang memperoleh lisensi dari lembaga negara yang berwenang memberikan lisensi terhadap lembaga sertifikasi profesi di Indonesia.",
    "Profil risiko Perusahaan mempertimbangkan antara lain letak geografis, produk Perusahaan, rencana bisnis, dan klaim experience dalam mengukur profil risiko serta mempertimbangkan hasil simulasi skenario perubahan (stress test). Selain itu Perusahaan dapat mempertimbangkan risiko lain yang mungkin timbul antara lain berasal dari rencana perubahan strategi dan/atau pengembangan bisnis Perusahaan.",
    "Memiliki pusat data dan melakukan rekam cadang data aktivitas yang diproses menggunakan Teknologi Informasi yang dilakukan secara berkala",
    "Megawati Soekarnoputri",
    "Modal disetor Bursa Efek sekurang-kurangnya berjumlah Rp7.500.000.000,00 (tujuh miliar lima ratus juta rupiah).",
    "Pihak yang diperiksa berhak meminta kepada Pemeriksa untuk memperlihatkan Surat Perintah Pemeriksaan dan Tanda Pengenal Pemeriksa, Pihak yang diperiksa berhak meminta kepada Pemeriksa untuk memberikan penjelasan tentang maksud dan tujuan pemeriksaan, dan Pihak yang diperiksa menandatangani surat pernyataan persetujuan tentang hasil pemeriksaan.",
    "Bagi permohonan izin usaha Perusahaan Asuransi dan Perusahaan Reasuransi yang telah diajukan dan yang telah memperoleh izin prinsip sebelum Peraturan Pemerintah ini ditetapkan, persyaratan permodalan tetap diberlakukan berdasarkan persyaratan yang berlaku pada saat izin prinsip ditetapkan.",
    "Rp. 2.000.000.000,-",
    "Bank Perkreditan Rakyat, yang selanjutnya disebut BPR, adalah Bank Perkreditan Rakyat sebagaimana dimaksud dalam Pasal 1 angka 4 Undang Undang Nomor 7 Tahun 1992 tentang Perbankan sebagaimana telah diubah dengan Undang-undang Nomor 10 Tahun 1998, yang melaksanakan kegiatan usaha secara konvensional.",
    "Dalam hal calon Pemegang Saham Pengendali BPR adalah pemerintah, maka pelaksanaan wawancara sebagaimana dimaksud dalam Pasal 8 ayat (1) huruf b hanya dilakukan apabila dianggap perlu.",
    "Modal disetor untuk mendirikan Bank ditetapkan sekurang-kurangnya sebesar Rp3.000.000.000.000,00 (tiga triliun rupiah).",
    "Bahwa dalam rangka meningkatkan good corporate governance, bank perlu menetapkan sasaran strategis dan seperangkat nilai perusahaan (corporate values) yang mengarahkan kegiatan operasional bank;",
    "PERATURAN BANK INDONESIA NOMOR 5/17/PBI/2003 TENTANG PERSYARATAN DAN TATA CARA PELAKSANAAN JAMINAN PEMERINTAH TERHADAP KEWAJIBAN PEMBAYARAN BANK PERKREDITAN RAKYAT",
    "Kepada SEMUA BANK UMUM YANG MELAKUKAN KEGIATAN USAHA SECARA KONVENSIONAL DI INDONESIA",
    "A. Penyusunan dan penyampaian Laporan Bulanan dan/atau Koreksi Laporan Bulanan untuk posisi bulan sebelum bulan Agustus 2013 tetap berpedoman pada Surat Edaran Bank Indonesia Nomor 8/7/DPBPR tanggal 23 Februari 2006 perihal Laporan Bulanan Bank Perkreditan Rakyat sebagaimana telah diubah terakhir dengan Surat Edaran Bank Indonesia Nomor 12/15/DKBU tanggal 11 Juni 2010. B. BPR Pelapor melakukan uji coba penyampaian Laporan Bulanan untuk posisi bulan Juni dan Juli 2013 yang masing-masing disampaikan paling lambat pada akhir bulan berikutnya, dengan berpedoman pada Surat Edaran Bank Indonesia ini.",
    "Dengan berlakunya Surat Edaran Bank Indonesia ini maka Surat Edaran Bank Indonesia Nomor 8/19/DPbS tanggal 24 Agustus 2006 perihal Pedoman Pengawasan Syariah dan Tata Cara Pelaporan Hasil Pengawasan bagi Dewan Pengawas Syariah dicabut dan dinyatakan tidak berlaku.",
    "Jakarta, 9 Juli 2013",
    "Bank Syariah wajib membentuk cadangan kerugian penurunan nilai (CKPN) atas aset keuangan dan aset non keuangan sesuai dengan standar akuntansi keuangan yang berlaku.",
    "Berdasarkan pertimbangan sebagaimana dimaksud angka I, maka penurunan Indeks Harga Saham Gabungan Bursa Efek Indonesia sejak tanggal 20 Mei 2013 sampai dengan tanggal 27 Agustus 2013 ini sebesar 1.247,134 poin atau 23,91% (dua puluh tiga koma sembilan satu perseratus) ditetapkan sebagai Kondisi Lain sebagaimana dimaksud Pasal 1 angka 1 huruf b POJK Nomor 2/POJK.04/2013.",
    "Pengguna S-INVEST yang bertindak sebagai Bank Kustodian melakukan uji coba penyampaian surat atau bukti konfirmasi dan laporan berkala Reksa Dana melalui S-INVEST dalam jangka waktu 12 (dua belas) bulan sejak ditetapkannya Surat Edaran Otoritas Jasa Keuangan ini",
    "SURAT EDARAN OTORITAS JASA KEUANGAN REPUBLIK INDONESIA NOMOR 1/SEOJK.04/2022 TENTANG TATA CARA PERMOHONAN PERSETUJUAN SEBAGAI PENYEDIA SISTEM PENYELENGGARAAN RAPAT UMUM PEMEGANG SAHAM SECARA ELEKTRONIK (E-RUPS)",
    "Risiko inheren atas risiko strategis, operasional, asuransi, kredit, pasar, likuiditas, hukum, kepatuhan, dan reputasi.",
    "Kriteria ini digunakan untuk menilai kemampuan lembaga pemeringkat dalam memberikan jasa pemeringkatan, baik dari aspek sumber daya manusia (human resources), aspek sumber daya keuangan (financial resources), maupun dukungan pemegang saham, yang memungkinkan lembaga pemeringkat beroperasi secara independen dan profesional.",
    "Asuransi atas risiko kegagalan Ekspor, atas risiko kegagalan bayar, atas investasi yang dilakukan oleh perusahaan Indonesia di luar negeri, dan/atau atas risiko politik di suatu negara yang menjadi tujuan ekspor",
    "Badan Supervisi terdiri 5 (lima) orang anggota terdiri dari seorang Ketua merangkap anggota, dan 4 (empat) orang anggota yang dipilih oleh Dewan Perwakilan Rakyat dan diangkat oleh Presiden untuk masa jabatan 3 (tiga) tahun dan dapat dipilih kembali untuk satu kali masa jabatan berikutnya.",
    "Berdasarkan Pasal 134 UU PPSK, Dana Pensiun adalah badan hukum yang mengelola dan menjalankan program yang menjanjikan manfaat pensiun. Selain manfaat pensiun, Dana Pensiun dapat menyelanggarakan manfaat lain sebagai manfaat tambahan sebagaimana diatur dalam peraturan perundang-undangan di bidang Dana Pensiun.",
    "Krisis keuangan akhir-akhir ini di Amerika Serikat yang merupakan terbesar sejak krisis 1929",
    "Soetikno Slamet"
]

examples = [
    {"query": q, "ground_truth": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [25]:
from langsmith import Client
from langsmith.utils import LangSmithError

client = Client()
dataset_name = "Chatbot OJK Langchain"

try:
    # check if dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("using existing dataset: ", dataset.name)
except LangSmithError:
    # if not create a new one with the generated query examples
    dataset = client.create_dataset(
        dataset_name=dataset_name, description="OJK Langchain test dataset"
    )
    for e in examples:
        client.create_example(
            inputs={"query": e["query"]},
            outputs={"ground_truth": e["ground_truth"]},
            dataset_id=dataset.id,
        )

    print("Created a new dataset: ", dataset.name)

using existing dataset:  Chatbot OJK Langchain


In [27]:
import time
from datasets import Dataset

results = []
contexts = []
queries = eval_questions
ground_truths = eval_answers

for query in queries:
    result = final_chain.invoke(
        {"question": query},
        config={
            "configurable": {"user_id": "xmriz", "conversation_id": "conv1"}
        },
    )

    results.append(result['answer'])
    sources = result["context"]
    final_sources = list(eval(sources))
    contents = []
    for source in final_sources:
        contents.append(source.get('page_content'))
    contexts.append(contents)

    time.sleep(2)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

In [28]:
print(chat_store.get_session_history(user_id="xmriz", conversation_id="conv1"))

Human: Asuransi dapat diberikan dalam bentuk apa saja?
AI: Asuransi dapat diberikan dalam bentuk: 
a. Asuransi atas risiko kegagalan Ekspor;
b. Asuransi atas risiko kegagalan bayar;
c. Asuransi atas investasi yang dilakukan oleh perusahaan Indonesia di luar negeri; dan/atau
d. Asuransi atas risiko politik di suatu negara yang menjadi tujuan ekspor.

Source: Undang-Undang Republik Indonesia tentang Lembaga Pembiayaan Ekspor Impor (UU no. 2 Tahun 2009)
URL: [Undang-Undang Republik Indonesia tentang Lembaga Pembiayaan Ekspor Impor](https://www.ojk.go.id/id/regulasi/Documents/Pages/UU-Republik-Indonesia-tentang-Lembaga-Pembiayaan-Ekspor-Impor/UU%20no.%202%20th.%202009%20ttg.%20LPEI.pdf#page=5)
Human: Badan Supervisi terdiri dari berapa anggota?
AI: Badan Supervisi terdiri dari 5 (lima) orang anggota.

Source: [3 Tahun 2004](https://www.ojk.go.id/id/regulasi/Documents/Pages/UU-Republik-Indonesia-tentang-Perubahan-atas-Undang-Undang-Republik-Indonesia-Nomor-23-Tahun-1999-tentang-Bank-Indones

In [29]:
import json

with open("dict_eval.json", "w") as outfile: 
    json.dump(d, outfile)

In [30]:
with open('dict_eval.json') as json_file:
    d = json.load(json_file)

In [31]:
dataset = Dataset.from_dict(d)

df = dataset.to_pandas()
df.head()

,question,answer,contexts,ground_truth
0,Apa itu SWDKLLJ?,SWDKLLJ adalah singkatan dari Sumbangan Wajib ...,[wajib\nsebagaimana dimaksud dalam Undang-Unda...,Sumbangan Wajib Dana Kecelakaan Lalu Lintas ad...
1,Bursa Efek wajib menyampaikan laporan realisas...,Bursa Efek wajib menyampaikan laporan realisas...,[13. Bursa Efek wajib menyampaikan laporan rea...,Bursa Efek wajib menyampaikan laporan realisas...
2,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,Isi dari KEPUTUSAN KETUA BADAN PENGAWAS PASAR ...,"[KETUA BADAN PENGAWAS PASAR MODAL,\nMenimbang ...",TATA CARA PEMBERIAN PERSETUJUAN ANGGARAN DASAR...
3,Siapa yang menandatangani keputusan ketua bape...,I PUTU GEDE ARY SUTA\n\nSource: KEP-07/PM/1996...,[Bapepam melakukan penelitian atas kelengkapan...,"I Putu Gede Ary Suta, Badan Pengawas Pasar Modal"
4,Penolakan atas permohonan persetujuan mengenai...,Formulir nomor yang digunakan untuk penolakan ...,[menggunakan Formulir Nomor III.B.2-2 lampiran...,Penolakan atas permohonan persetujuan mengenai...


In [32]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate
import nest_asyncio
nest_asyncio.apply()

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)

Evaluating: 100%|██████████| 315/315 [01:55<00:00,  2.73it/s]


In [33]:
df = score.to_pandas()
df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,answer_similarity,answer_correctness,harmfulness
0,Apa itu SWDKLLJ?,SWDKLLJ adalah singkatan dari Sumbangan Wajib ...,[wajib\nsebagaimana dimaksud dalam Undang-Unda...,Sumbangan Wajib Dana Kecelakaan Lalu Lintas ad...,1.000000,0.960398,1.000000,1.000000,0.912987,0.603247,0
1,Bursa Efek wajib menyampaikan laporan realisas...,Bursa Efek wajib menyampaikan laporan realisas...,[13. Bursa Efek wajib menyampaikan laporan rea...,Bursa Efek wajib menyampaikan laporan realisas...,1.000000,0.952500,1.000000,1.000000,0.960775,0.990194,0
2,KEPUTUSAN KETUA BADAN PENGAWAS PASAR MODAL NOM...,Isi dari KEPUTUSAN KETUA BADAN PENGAWAS PASAR ...,"[KETUA BADAN PENGAWAS PASAR MODAL,\nMenimbang ...",TATA CARA PEMBERIAN PERSETUJUAN ANGGARAN DASAR...,0.000000,0.960082,0.555556,1.000000,0.897958,0.724490,0
3,Siapa yang menandatangani keputusan ketua bape...,I PUTU GEDE ARY SUTA\n\nSource: KEP-07/PM/1996...,[Bapepam melakukan penelitian atas kelengkapan...,"I Putu Gede Ary Suta, Badan Pengawas Pasar Modal",0.000000,0.846042,0.950000,1.000000,0.860175,0.715044,0
4,Penolakan atas permohonan persetujuan mengenai...,Formulir nomor yang digunakan untuk penolakan ...,[menggunakan Formulir Nomor III.B.2-2 lampiran...,Penolakan atas permohonan persetujuan mengenai...,1.000000,0.971007,1.000000,1.000000,0.965531,0.741383,0
5,Anggota Dewan Pengawas Syariah Perusahaan Pemb...,Anggota Dewan Pengawas Syariah Perusahaan Pemb...,"[yakni Pasal 10A, sehingga berbunyi sebagai be...",Dilarang melakukan rangkap jabatan sebagai ang...,1.000000,0.956593,1.000000,1.000000,0.951977,0.737994,0
6,Berapa uang pertanggungan dan pertanggungan to...,Uang pertanggungan untuk kategori 5 adalah leb...,[PERTANGGUNGAN \nTOTAL LOSS ONLY \n(TLO) \nPER...,"Lebih dari Rp800.0000.000,00 dan 0,28%",0.000000,0.901098,1.000000,1.000000,0.835920,0.708980,0
7,Perusahaan wajib menyampaikan laporan hasil pe...,Perusahaan wajib menyampaikan laporan hasil pe...,[laporan hasil pengawasan Dewan Pengawas Syari...,Perusahaan wajib menyampaikan laporan hasil pe...,0.250000,0.926545,0.966667,1.000000,0.986976,0.792198,0
8,Batas tingkat solvabilitas minimum bagi perusa...,Batas tingkat solvabilitas minimum bagi perusa...,[metadata={'title': 'Peraturan BAPEPAM tentang...,Batas tingkat solvabilitas minimum bagi perusa...,0.000000,0.927443,1.000000,1.000000,0.935836,0.233959,0
9,Kapan PERATURAN KETUA BADAN PENGAWAS PASAR MOD...,(Answer...) \n\nTidak ada informasi mengenai P...,[DAN LEMBAGA KEUANGAN NOMOR PER-03/BL/2007 TE...,Peraturan Ketua ini mulai berlaku pada tanggal...,0.000000,0.000000,0.926667,0.000000,0.816225,0.204056,0


In [34]:
for key,val in score.items():
    print(f"{key}: {val}")

faithfulness: 0.6265432098765432
answer_relevancy: 0.8344475315305905
context_precision: 0.8215493826874766
context_recall: 0.8544444444444446
answer_similarity: 0.9000963850383613
answer_correctness: 0.5542750457532194
harmfulness: 0.0
